In [2]:
text = (
       'Hello, how are you? I am Romeo.\n'
       'Hello, Romeo My name is Juliet. Nice to meet you.\n'
       'Nice meet you too. How are you today?\n'
       'Great. My baseball team won the competition.\n'
       'Oh Congratulations, Juliet\n'
       'Thanks you Romeo'
   )

   

In [3]:
import re
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))

In [4]:
token_list = [i.split() for i in sentences]
token_list

[['hello', 'how', 'are', 'you', 'i', 'am', 'romeo'],
 ['hello', 'romeo', 'my', 'name', 'is', 'juliet', 'nice', 'to', 'meet', 'you'],
 ['nice', 'meet', 'you', 'too', 'how', 'are', 'you', 'today'],
 ['great', 'my', 'baseball', 'team', 'won', 'the', 'competition'],
 ['oh', 'congratulations', 'juliet'],
 ['thanks', 'you', 'romeo']]

In [5]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3,0:0,1:1,2:2,3:3}
for i,w in enumerate(word_list):
   word_dict[w] = i + 4
   number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)

In [6]:
vocab_size,number_dict

(33,
 {0: '[PAD]',
  1: '[CLS]',
  2: '[SEP]',
  3: '[MASK]',
  4: 0,
  5: 1,
  6: 2,
  7: 3,
  8: 'how',
  9: 'too',
  10: 'to',
  11: 'thanks',
  12: 'juliet',
  13: 'you',
  14: 'baseball',
  15: 'congratulations',
  16: 'i',
  17: 'team',
  18: 'oh',
  19: 'romeo',
  20: 'is',
  21: 'name',
  22: 'competition',
  23: 'the',
  24: 'today',
  25: 'great',
  26: 'are',
  27: 'meet',
  28: 'hello',
  29: 'nice',
  30: 'am',
  31: 'won',
  32: 'my'})

In [7]:
from random import randrange,shuffle,randint
tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences)) 
tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] #token embeeing kind
input_ids =  [word_dict[i] for i in input_ids if i in word_dict] 
segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)


segment_ids,input_ids


([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 14, 11, 8, 2, 24, 4, 22, 9, 12, 26, 15, 2])

In [8]:
tokens_b

['hello', 'how', 'are', 'you', 'i', 'am', 'romeo']

In [9]:
max_pred = 8
n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15))))


In [10]:
cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
shuffle(cand_maked_pos)
cand_maked_pos

[8, 7, 9, 5, 6, 1, 2, 11, 10, 3]

In [11]:
masked_tokens, masked_pos = [], []
import random
rand = random.Random()
for pos in cand_maked_pos[:n_pred]:  
           masked_pos.append(pos)
           masked_tokens.append(input_ids[pos])
           ran1 = rand.uniform(0.5,0.8)
           if   ran1< 0.8 and ran1>0.5:  # 80%
               input_ids[pos] = word_dict['[MASK]'] # make mask?
               print(input_ids)
           elif rand.uniform(0,1) < 0.5:  # 10%-----> replacing one word with random word...
               print("hello i am masking")
               index = randint(0, vocab_size - 1) # random index in vocabulary
               input_ids[pos] = word_dict[number_dict[index]] # replace

[1, 14, 11, 8, 2, 24, 4, 22, 3, 12, 26, 15, 2]
[1, 14, 11, 8, 2, 24, 4, 3, 3, 12, 26, 15, 2]


In [12]:
maxlen = 128
n_pad = maxlen - len(input_ids)
input_ids.extend([0] * n_pad)
segment_ids.extend([0] * n_pad)
if max_pred > n_pred:
           n_pad = max_pred - n_pred
           masked_tokens.extend([0] * n_pad)
           masked_pos.extend([0] * n_pad)

In [13]:
segment_ids = segment_ids[:128]

In [14]:
from random import randrange,shuffle,randint
import random

In [15]:
import random
from random import randrange,shuffle
batch_size = 2
def make_batch():
   batch = []
   positive = negative = 0
   while positive != batch_size/2 or negative != batch_size/2:#getting half -ve and half positive sentence 
       tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences)) 
       
       tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]
    #    print(tokens_a,tokens_b)

       input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] #token embeeing kind
       input_ids =  [word_dict[i] for i in input_ids if i in word_dict] 
       segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
 
       # MASK LM
       cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
       shuffle(cand_maked_pos)
       cand_maked_pos
       
       masked_tokens, masked_pos = [], []
       import random
       rand = random.Random()
       for pos in cand_maked_pos[:n_pred]:  
           masked_pos.append(pos)
           masked_tokens.append(input_ids[pos])
           ran1 = rand.uniform(0.5,0.8)
           if   ran1< 0.8 and ran1>0.5:  # 80%
               input_ids[pos] = word_dict['[MASK]'] # make mask?
               #print(input_ids)
           elif rand.uniform(0,1) < 0.5:  # 10%-----> replacing one word with random word...
               #print("hello i am masking")
               index = randint(0, vocab_size - 1) # random index in vocabulary
               input_ids[pos] = word_dict[number_dict[index]] # replace
        # """
        # from line 25 to 32 we are adding mask to our vocabs
        # """

       # Zero Paddings
       maxlen = 128
       n_pad = maxlen - len(input_ids)
       input_ids.extend([0] * n_pad)
       segment_ids.extend([0] * n_pad)
       if max_pred > n_pred:
           n_pad = max_pred - n_pred
           masked_tokens.extend([0] * n_pad)
           masked_pos.extend([0] * n_pad)
        
        #i got my sentence to my neural net ready...2 sent to 1 sent with pad,cls,slp etc tokens
        #and i get segment ids, maked tokens, maskes token positons with padding to max size

        #"""
        # if (3,2 is the pair) then it falls into negative sent as sent 2 dint come next to sent 3. but if (2,3) then its a positive sentece
        #"""
       if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
           positive += 1
       elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
           batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
           negative += 1
       return batch

In [16]:
btch = make_batch()

In [103]:
btch[0][3]

[8, 13, 0, 0, 0, 0, 0, 0]

In [17]:
import torch

print(torch.arange(128, dtype=torch.long).expand_as(torch.tensor(input_ids)))


tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127])


In [129]:
input_ids

tensor([[ 1, 28, 19,  3,  5, 24,  4, 18,  2, 28,  3, 15, 11, 27, 17, 20, 25,  8,
          5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])

# NN WORK


In [18]:
from torch import nn
d_model = 264
n_segments = 2
class Embedding(nn.Module):
   def __init__(self):
       super(Embedding, self).__init__()
       self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
       self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
       self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
       self.norm = nn.LayerNorm(d_model)

   def forward(self, x, seg):
       seq_len = x.size(1)
       pos = torch.arange(seq_len, dtype=torch.long)
       #print(pos.shape,x.shape,seg.shape)
       pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
       embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
       return self.norm(embedding)

In [117]:
emb_nn = Embedding()
input_ids,segment_ids = torch.tensor(input_ids).view(1,-1),torch.tensor(segment_ids).view(1,-1)
enc_input = emb_nn(torch.tensor(input_ids),torch.tensor(segment_ids))

<ipython-input-117-89a70f0cb947>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids,segment_ids = torch.tensor(input_ids).view(1,-1),torch.tensor(segment_ids).view(1,-1)
<ipython-input-117-89a70f0cb947>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  enc_input = emb_nn(torch.tensor(input_ids),torch.tensor(segment_ids))


In [83]:
def get_attn_pad_mask(seq_q, seq_k):#for padding purposes
   batch_size, len_q = seq_q.size()
   batch_size, len_k = seq_k.size()
   # eq(zero) is PAD token
   pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
   return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k-->batch_size x no_heads x len_q x len_k
print(get_attn_pad_mask(input_ids, input_ids)[0][0], input_ids[0])
#print(tor)
attn_mask = get_attn_pad_mask(input_ids, input_ids)[0][0]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])

In [169]:
class EncoderLayer(nn.Module):
   def __init__(self):
       super(EncoderLayer, self).__init__()
       self.enc_self_attn = MultiHeadAttention()
       self.pos_ffn = PoswiseFeedForwardNet()

   def forward(self, enc_inputs, enc_self_attn_mask):
       enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
       enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
       return enc_outputs, attn

These three inputs and the attention mask are operated with a dot product operation that yields two outputs: 
context vectors and attention. The context vector is then passed through a linear layer and finally that yields the output.



In [168]:
from sklearn.metrics import SCORERS


n_heads,d_k,d_v = 8,64,64
class MultiHeadAttention(nn.Module):
   def __init__(self):
       super(MultiHeadAttention, self).__init__()
       self.W_Q = nn.Linear(d_model, d_k * n_heads) #42--->7...6 heads---> 42
       self.W_K = nn.Linear(d_model, d_k * n_heads)
       self.W_V = nn.Linear(d_model, d_v * n_heads) #element wise product

   def forward(self, Q, K, V, attn_mask):
        #Q, K, V,-->seq input sentence--->[batch,seq_l,d_model]
       # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
       residual, batch_size = Q, Q.size(0)
       # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
       q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
       k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
       v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
       #attn_mask-->batch_s*num_seq
       attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]
       # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       context, attn,scores = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
       context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
       output = nn.Linear(n_heads * d_v, d_model)(context)
       return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]

In [37]:
arr = torch.tensor(([-1e5,14,12,34,11,0.1,0]))
nn.Softmax()(arr)


<ipython-input-37-55e8e42b2bf0>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.Softmax()(arr)


tensor([0.0000e+00, 2.0612e-09, 2.7895e-10, 1.0000e+00, 1.0262e-10, 1.8942e-15,
        1.7139e-15])

In [79]:
import  numpy as np
class ScaledDotProductAttention(nn.Module):
   def __init__(self):
       super(ScaledDotProductAttention, self).__init__()
   def forward(self, Q, K, V, attn_mask):
       scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       #################
       
       scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
       ###why this upper line...scores..(why -1e9)
       attn = nn.Softmax(dim=-1)(scores)

       context = torch.matmul(attn, V)  
       return context, attn,scores
       

In [88]:
scores1 = score
attn_mask
scores1.masked_fill_(attn_mask, -1e9)[0][0][0][:40]

tensor([-4.8446e-01, -1.3233e-01, -5.7382e-01, -3.8168e-01,  2.7246e-03,
         5.5001e-02, -5.2757e-01, -1.0408e-01, -6.6770e-02, -2.8307e-01,
        -9.5595e-02,  1.5389e-01,  2.0026e-01, -1.0000e+09, -1.0000e+09,
        -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
        -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
        -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
        -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
        -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
       grad_fn=<SliceBackward>)

In [90]:
attention = MultiHeadAttention()
attn_output,att,score = attention(enc_input,enc_input,enc_input,attn_mask)

In [91]:

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
      super(PoswiseFeedForwardNet, self).__init__() #264
      self.fc1 = nn.Linear(d_model,d_model)
      self.norm = nn.LayerNorm(d_model)
    def forward(self,ip):
      return self.norm(self.fc1(ip)+ip)
ffn = PoswiseFeedForwardNet()
ffn(attn_output).shape

torch.Size([1, 128, 264])

In [167]:
!git clone https://github.com/ashishgupta2598/Study-AI.git

Cloning into 'Study-AI'...


In [92]:
import torch
x = torch.randn(2, 3,5,6,7)
x = torch.transpose(x, 1, 4)
x.shape
x = torch.tensor([[1, 2, 3],[7,8,9]])
print(x.shape)
x.repeat(4,3,2,2).shape

torch.Size([2, 3])


torch.Size([4, 3, 4, 6])

In [93]:
from torch import Tensor
import torch.nn.functional as F

class GELU(nn.Module):
    def forward(self, input: Tensor) -> Tensor:
        return F.gelu(input)
import math
def gelu(x):
   return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


In [170]:
n_layers = 12
class BERT(nn.Module):
   def __init__(self):
       super(BERT, self).__init__()
       self.embedding = Embedding()
       self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
       self.fc = nn.Linear(d_model, d_model)
       self.activ1 = nn.Tanh()
       self.linear = nn.Linear(d_model, d_model)
       self.activ2 = GELU()
       self.norm = nn.LayerNorm(d_model)
       self.classifier = nn.Linear(d_model, 2) #weather its next sentence or not of first sentence

       # decoder is shared with embedding layer
       embed_weight = self.embedding.tok_embed.weight
       n_vocab, n_dim = embed_weight.size()
       self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
       self.decoder.weight = embed_weight
       self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

   def forward(self, input_ids, segment_ids, masked_pos):
       output = self.embedding(input_ids, segment_ids)
       enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
       for layer in self.layers:
           output, enc_self_attn = layer(output, enc_self_attn_mask)
       # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
       # it will be decided by first token(CLS)
       h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
       logits_clsf = self.classifier(h_pooled) # [batch_size, 2]


        ###final masked positions
        
       masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]

       # get masked position from final output of transformer.
       h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
       h_masked = self.norm(self.activ2(self.linear(h_masked)))
       logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

       return logits_lm, logits_clsf

In [171]:
enc_input.shape

torch.Size([1, 128, 264])

In [127]:
masked_pos.shape

torch.Size([1, 8, 264])

In [122]:
# masked_pos = mk[:, :, None].expand(-1, -1, 264)

In [132]:
enc_input.shape

torch.Size([1, 128, 264])

In [144]:
arr = []
for i in mk[0]:
    print(i)
    arr.append(enc_input[0][i])
arr1 = torch.stack(arr)

tensor(8)
tensor(13)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [145]:
arr1

tensor([[-0.7482, -0.6406,  2.3126,  ...,  1.3393, -1.0581, -1.0358],
        [-0.5056,  1.3264,  2.1279,  ...,  0.5359, -1.7751,  0.8639],
        [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
        ...,
        [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
        [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
        [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545]],
       grad_fn=<StackBackward>)

In [147]:
torch.gather(enc_input, 1, masked_pos) # masking position [batch_size, max_pred, d_model]


tensor([[[-0.7482, -0.6406,  2.3126,  ...,  1.3393, -1.0581, -1.0358],
         [-0.5056,  1.3264,  2.1279,  ...,  0.5359, -1.7751,  0.8639],
         [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
         ...,
         [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
         [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545],
         [ 0.7458,  0.9808,  1.7282,  ..., -0.4824, -0.9228,  0.8545]]],
       grad_fn=<GatherBackward>)

In [150]:
enc_input.shape

torch.Size([1, 128, 264])

In [153]:
masked_pos[0][1]

tensor([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 

In [ ]:
for i in 

In [109]:
mk = torch.tensor([btch[0][3]])

In [112]:
mk

tensor([[ 8, 13,  0,  0,  0,  0,  0,  0]])

In [164]:
model = BERT()

In [172]:
criterion = nn.CrossEntropyLoss()
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [173]:
model = BERT()
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
print(input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext)

torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 8]) torch.Size([1, 8]) tensor([1])


In [174]:
for epoch in range(1121):
       optimizer.zero_grad()
    #    batch = make_batch()
    #    input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
       logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
       loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
       loss_lm = (loss_lm.float()).mean()
       loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
       loss = loss_lm + loss_clsf
       if (epoch + 1) % 10 == 0:
           print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
       loss.backward()
       optimizer.step()

       


Epoch: 0010 cost = 36.806892
Epoch: 0020 cost = 44.657158
Epoch: 0030 cost = 26.927658
Epoch: 0040 cost = 51.918751
Epoch: 0050 cost = 37.517300
Epoch: 0060 cost = 38.391052
Epoch: 0070 cost = 46.027973
Epoch: 0080 cost = 40.382893
Epoch: 0090 cost = 72.925652
Epoch: 0100 cost = 39.761505
Epoch: 0110 cost = 50.296665
Epoch: 0120 cost = 53.890007
Epoch: 0130 cost = 44.961403
Epoch: 0140 cost = 55.985207
Epoch: 0150 cost = 64.352257


KeyboardInterrupt: 

In [166]:
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print("hello-------ṭ ",text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

hello-------ṭ  Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
['[CLS]', 'won', 'am', 1, 'you', '[MASK]', 'too', 1, 'meet', '[SEP]', 'i', 'my', 'too', 1, '[MASK]', 0, 'great', '[SEP]']
masked tokens list :  [19, 24]
predict masked tokens list :  [10, 30, 27, 27, 27, 27, 27, 27]
isNext :  False
predict isNext :  True
